In [ ]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
import sys
# reload(sys)
# sys.setdefaultencoding('utf8')
import os
import json
import string
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import random
from numpy import array
import re
import pickle
#from pattern.text.en import singularize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
#import enchant
from nltk.stem import LancasterStemmer
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
from sklearn.metrics import f1_score
import operator
from collections import Counter
from nltk.corpus import wordnet as wn
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from collections import defaultdict
import math
import csv

import pandas as pd 
import numpy as np 
from collections import defaultdict
import re
import sys
from sklearn.model_selection import train_test_split
import gzip

import csv
import pandas as pd 
import numpy as np 
from collections import defaultdict
import re
import sys
from sklearn.model_selection import train_test_split
import datetime
import matplotlib.pyplot as plt
import time
import numpy as np
from pprint import pprint

import spacy  # For preprocessing
import nltk
import string
from nltk.corpus import stopwords
import preprocessor as p  #pip install tweet-preprocessor
import logging  # Setting up the loggings to monitor gensim
from nltk.stem import WordNetLemmatizer
from string import punctuation as punc

import gensim
import gensim.corpora as corpora
#from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.models as gsm
from gensim.test.utils import datapath

import regex


import torch
import torchtext
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import torchvision.transforms as transforms
from torch.autograd import Variable

import spacy
import string

from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
from nltk.corpus import stopwords

torch.manual_seed(1)





In [ ]:
stop_words=['hundred', 'thousand', 'news', 'daily', 'zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'per', 'a', 'an',\
                   'the', 'and', 'but', 'or', 'should', 'would', 'might', 'could', 'of', 'with', 'at', 'from', 'into', 'during', 'including', 'until', 'against',\
                   'among', 'throughout', 'despite', 'towards', 'upon', 'concerning', 'to', 'in', 'for', 'on', 'by', 'about', 'like', 'through', 'over', 'before',\
                   'between', 'after', 'since', 'without', 'under', 'within', 'along', 'following', 'across', 'behind', 'beyond', 'plus', 'except', 'but', 'up',\
                   'out', 'around', 'down', 'off', 'above', 'near', 'r',\
                    'today','tomorrow','yesterday','todays','everyday','m','s','re']


In [ ]:
utypes = {'utype_prac' : 0, 'utype_promo' : 1, 'utype_none' : -1}
reverse_utypes = {'0' : 'utype_prac', '1' : 'utype_promo', '-1' : 'utype_none'}


In [ ]:
#get stop words of all languages
STOPWORDS_DICT = {lang: set(nltk.corpus.stopwords.words(lang)) for lang in nltk.corpus.stopwords.fileids()}
#get english stopwords
en_stopwords = set(stopwords.words('english'))
#load spacy

nlp = spacy.load('en')

In [ ]:
#function to clean and lemmatize ads
def clean_ads(text):
    #print(text)
    text = re.sub(r"http\S+", "", text) #delete http
    text = re.sub(r"www\S+", "", text)
    text = re.sub("\d", "", text) #remove digit
    text = re.sub(r'[^\w\d\s]+', '', text) #remove punctuations
    #print(text)
    #remove punctuations
#     regex = re.compile('[' + re.escape(string.punctuation) + '\\r\\t\\n]')   
#     nopunct = regex.sub(" ", str(text))
    #print(nopunct)
    
    #use spacy to lemmatize comments
    #doc = nlp(nopunct, disable=['parser','ner'])
    doc = nlp(text, disable=['parser','ner'])
    #lemma = [token.lemma_ for token in doc]
    lemma = [token.lemma_.lower() if token.lemma_ != '-PRON-' else token.lower_ for token in doc]  #remove '-PRON-'
    return lemma

In [ ]:
#function to filter for ADJ/NN bigrams
def rightTypes(ngram):
    if '-pron-' in ngram or '' in ngram or ' 'in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in en_stopwords or word in stop_words:
            return False
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    second_type = ('NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False

In [ ]:
def rightTypesTri(ngram):
    if '-pron-' in ngram or '' in ngram or ' 'in ngram or '  ' in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in en_stopwords or word in stop_words:
            return False
    first_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    third_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in first_type and tags[2][1] in third_type:
        return True
    else:
        return False

In [ ]:

def get_ngrams(unlist_ads):
    ##Initialize NLTK's Bigrams/Trigrams Finder
    bigrams = nltk.collocations.BigramAssocMeasures()
    trigrams = nltk.collocations.TrigramAssocMeasures()
    bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(unlist_ads)
    trigramFinder = nltk.collocations.TrigramCollocationFinder.from_words(unlist_ads)

    ##frequency based
    ##Counting Frequencies of Adjacent Words filter for only adjectives and nouns
    bigram_freq = bigramFinder.ngram_fd.items()
    bigramFreqTable = pd.DataFrame(list(bigram_freq), columns=['bigram','freq']).sort_values(by='freq', ascending=False)
    #filter bigrams
    filtered_bi = bigramFreqTable[bigramFreqTable.bigram.map(lambda x: rightTypes(x))]
    #print("filtered_bi", len(filtered_bi))
    ## trigram
    trigram_freq = trigramFinder.ngram_fd.items()
    trigramFreqTable = pd.DataFrame(list(trigram_freq), columns=['trigram','freq']).sort_values(by='freq', ascending=False)
    #filter trigrams
    filtered_tri = trigramFreqTable[trigramFreqTable.trigram.map(lambda x: rightTypesTri(x))]
    #print("filtered_tri", len(filtered_bi), len(filtered_tri))
    #print(filtered_tri)
    ##if we choose freq >= 2
    #new_filtered_tri = filtered_tri.loc[filtered_tri['freq'] >= 2]
    #print(new_filtered_tri)
    #print("filtered_tri", len(filtered_tri), len(new_filtered_tri))
    freq_bi= filtered_bi[:].bigram.values 
    freq_tri = filtered_tri[:].trigram.values 
    
    ##convert comma separated trigram list to "_" separated trigram
    #print(str(freq_tri[0]).replace("'","").replace(" ","").replace(",","_").replace("(","").replace(")",""))
    new_freq_bi = [str(trig).replace("'","").replace(" ","").replace(",","_").replace("(","").replace(")","") for trig in [trigs for trigs in freq_bi]]
    new_freq_tri = [str(trig).replace("'","").replace(" ","").replace(",","_").replace("(","").replace(")","") for trig in [trigs for trigs in freq_tri]]
    
    #print(new_freq_tri, len(new_freq_tri), type(new_freq_tri))
    return new_freq_bi, new_freq_tri

In [ ]:
def Read_CSV():
    csv.field_size_limit(sys.maxsize)
    name2text={} #user_name -> tweets
    name2loc={} #user_name -> location
    name2des={} #user_name -> description
    name2utype={} ##user_name -> utype
    name2utype_gt = {} ##user_name -> utype_gt
    des2utype={} #description -> utype 
    loc2utype={} #location -> utype 

    row_count=-1
    
        
    #with open("data/weakly_all_des_gt_mergetweets_yoga_13k_em1_des_net.csv", "r") as csv_file: #for em1
    with open("data/weakly_all_des_gt_mergetweets_yoga_13k_em2_des_net.csv", "r") as csv_file: #for em2
    #with open("data/weakly_all_des_gt_mergetweets_yoga_13k_em3_des_net.csv", "r") as csv_file: #for em3
    #with open("data/weakly_all_des_gt_mergetweets_yoga_13k_em4_des_net.csv", "r") as csv_file: #for em4
    
        csv_reader = csv.reader(csv_file, delimiter=str(','), quotechar=str('"'))
        for row in csv_reader:
            # print (row_count) #upto 13k ads
            #print(row)
            row_count += 1
            if row_count==0:
                continue

            name=str(row[0])
            loc = str(row[1]).lower()
            #print('loc', len(loc))
            des=str(row[2]).lower()
            #print('des', len(des))
            #utype = str(row[4]) #before em1
            #utype = str(row[6]) #for em1
            utype = str(row[7]) #for em2
            #utype = str(row[8]) #for em3
            #utype = str(row[9]) #for em4
            #print('utype', utype, type(utype))
            utype_gt = str(row[5]) #gold
            twt=str(row[3])
            text=twt

            #print(text)
            
            
            lemmatized = clean_ads(str(text))
            #print(lemmatized)
            # #turn all tokens into one single list
            unlist_ads = [item for item in lemmatized]
            # using join() +  split() to 
            # perform removal of empty string
            unlist_ads = ' '.join(unlist_ads).split() 
            #print(unlist_ads)
            if unlist_ads and len(unlist_ads) >= 1 : #if twt length is >= 1
                
                #new_freq_bi, new_freq_tri = get_ngrams(unlist_ads)
                #print(new_freq_bi)
#                 utype = str (row[4])
#                 utype_gt = str(row[5]) #gold

                name2text[name]=unlist_ads ##user_name -> tweets
                name2loc[name]= loc #user_name -> location
                name2des[name]= des #user_name -> description
                name2utype[name]= utype.split('.')[0] #utype##user_name -> utype
                name2utype_gt[name] = utype_gt.split('.')[0] ##user_name -> utype_gt
            if utype:
                des2utype[des] = utype.split('.')[0]#int(float(utype))  #description -> utype 
                loc2utype[loc] = utype.split('.')[0] #int(float(utype)) #location -> utype 
#             if des in des2utype:
#                 print (des, utype) 
#             des2utype[des] = utype


    print (len(name2text), len(des2utype), len(loc2utype)) #13301 13178 7302
    



    with open('data/graph_embd_yoga/processed_data_des_net_em2.pickle', 'wb') as out_file:
        pickle.dump([name2text, name2loc, name2des, name2utype, name2utype_gt, des2utype, loc2utype], out_file)







In [ ]:
Read_CSV() ###create pickle file for processed data (13k ads)

In [ ]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)


In [ ]:

glove={}
def ProcessGlove():
    global glove
    print ("Loading Glove Model")
    f = open("data/glove.6B.300d.txt", 'r')
    #f = open("../../Community_Detection/Resources/glove.twitter.27B/glove.twitter.27B.200d.txt", 'r')
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        glove[word] = embedding
    print ("Done.", len(glove), " words loaded!")

In [ ]:
def Tokenize_sentences(id2text, name2id):
    ad2tokenized_text={}
    idx2word={}
    word2idx={}
    word2count={}

    word2idx['<pad>'] = len(word2idx)
    idx2word[len(idx2word)] = '<pad>'
    for segment_id in id2text:
        tokenized_text=[]
        text = id2text[segment_id]
        #print(text)
#         text = text.lower()
#         text = text.split()
        for w in text:

            '''
            if w in stopwords.words('english'):
                continue
            '''
            #if w!="" and w not in word2idx and is_ascii(w) and w.isdigit() == False:
            if w!="" and w not in word2idx and w.isdigit() == False:
                word2idx[w]=len(word2idx)
                idx2word[len(idx2word)]=w

            '''
            if w!="" and is_ascii(w):
                tokenized_text.append(w)
            '''
            if w in word2idx:
                tokenized_text.append(word2idx[w])
            #if w!="" and is_ascii(w) and w.isdigit() == False and w in word2count:
            if w!="" and w.isdigit() == False and w in word2count:
                word2count[w]=word2count[w]+1
            #elif w!="" and is_ascii(w) and w.isdigit() == False and w not in word2count:
            elif w!="" and w.isdigit() == False and w not in word2count:
                word2count[w]=1
        ad2tokenized_text[name2id[segment_id]]=tokenized_text
        #print tokenized_text
        if len(tokenized_text)==0:
            print (text)

    print ("Tokenization complete!")
    ProcessGlove()
    #glove = torchtext.vocab.GloVe(name=\"6B\",dim=300)   # embedding size = 300
#     glove = torchtext.vocab.GloVe(name="6B", # trained on Wikipedia 2014 corpus\n"
#                                   dim=300)   # embedding size = 300"
   
#     print('first', glove[idx2word[0]])  
#     print('second', glove[idx2word[1]])
    #sys.exit()
    matrix_len = len(idx2word)
    weights_matrix = np.zeros((matrix_len, 300))
    words_found = 0
    not_found=[]
    for idx in idx2word:
#         print("idx2word", len(idx2word)) #44386
#         print("idx", idx) #idx 0
#         print("idx2word[idx]", idx2word[0], idx2word[1]) ### <pad> , trump
        try:
            weights_matrix[idx] = glove[idx2word[idx]]
            words_found += 1
        except KeyError:
            not_found.append(idx2word[idx])
            if idx==0:
                weights_matrix[idx] = np.random.normal(scale=0, size=(300,))
            else:
                weights_matrix[idx] = np.random.normal(scale=1/float(300), size=(300,))

    print ("not found: ", len(not_found), "found: ", words_found) #not found:  218162 found:  47851
    print (not_found[:10])

    single_count=0
    for w in not_found:
        if w=='<pad>':
            continue
        if word2count[w]==1:
            single_count+=1
    print ("single_count", single_count) #153485 #162228

    return ad2tokenized_text, weights_matrix

In [ ]:
def Create_Graph():

    with open('data/graph_embd_yoga/processed_data_des_net_em2.pickle', 'rb') as in_file:
        [name2text, name2loc, name2des, name2utype, name2utype_gt, des2utype, loc2utype]=pickle.load(in_file)
        
    with open('data/graph_embd_yoga/mention_yoga.pickle', 'rb') as in_file:
        [name2mention]=pickle.load(in_file)


#     name2text['prac']='yoga fan'
#     name2text['prac']='yoga enthusiast'
#     name2text['promo']='studio that shape your life'
#     name2text['promo']='book your free class'



    all_types=[typ for typ in utypes]
    #print(all_types, type(all_types)) #['type_prac', 'type_promo', 'type_none'] <class 'list'>
    all_users=[name for name in name2loc] 
    #print(all_users, type(all_users)) #['000813_000606', '000mrs000', '001bhaskarkumar'..], list
    all_descriptions=list(set([name2des[name] for name in name2des]))
    print("all_descriptions", type(all_descriptions), len(all_descriptions), len(name2des))
    all_locations=list(set([name2loc[name] for name in name2loc]))
    print('all_locations', len(all_locations), len(name2loc))

    
    name2id={}
    id2name={}
    id=0
    type_start=id
    for typ in all_types:
        name2id[typ]=id
        id2name[id]=typ
        id+=1
    type_end=id-1

#     id=10
#     indicator_label_start = id
#     for il in all_indicator_labels:
#         name2id[il] = id
#         id2name[id] = il
#         id += 1
#     indicator_label_end = id - 1

#     id=20
#     issue_start = id
#     for issue in all_issues:
#         name2id[issue] = id
#         id2name[id] = issue
#         id += 1
#     issue_end = id - 1

    id=100000
    user_start = id
    for user in all_users:
        name2id[user] = id
        id2name[id] = user
        id += 1
    user_end = id - 1

    id=1000000
    des_start = id
    for des in all_descriptions:
        name2id[des] = id
        id2name[id] = des
        id += 1
    des_end = id - 1
    
    id=2000000
    loc_start = id
    for loc in all_locations:
        name2id[loc] = id
        id2name[id] = loc
        id += 1
    loc_end = id - 1
    

    graph={}

    for id in id2name:
        graph[id]=set()
    #print(graph) #{0: set(), 1: set(), ......}
    

    '''
    user to description adjacency
    '''
    for user in all_users:
        if user not in name2des:
            continue
        des = [name2des[user]]
        des = [name2id[d] for d in des]

        graph[name2id[user]] = graph[name2id[user]] | set(des)
    #print(graph)  
    

    
    '''
    description to utype adjacency
    '''
    annotated_des=[]
    for des in all_descriptions:
        if des in des2utype:
#             print('des', des)
#             #print(des2utype)
#             print('des2utype[des]', des2utype[des], type(des2utype[des]))
#             print('reverse_utypes[des2utype[des]]', reverse_utypes[des2utype[des]])
            types=name2id[reverse_utypes[des2utype[des]]]
            graph[name2id[des]]=graph[name2id[des]] | set([types])
            annotated_des.append(name2id[des])
    #print(annotated_des, len(annotated_des))
    #print(graph) #{...1000191: {1}, 1000192: set()...}
    

    
    '''
    user to utype adjacency
    '''
    annotated_user=[]
    for id in name2utype:
        #print(id)
        #print(name2utype[id], type(name2utype[id]), len(name2utype[id]))
        for ty in name2utype[id]:
            if ty:
                #print('hi')
                #print(reverse_stances[st], type(reverse_stances[st])) #stance_anti-trump 
                utype=name2id[reverse_utypes[ty]] 
                #print(stance) #3
                graph[name2id[id]]=graph[name2id[id]] | set([utype])
                annotated_user.append(name2id[id])
        #graph[name2id[id]]=graph[name2id[id]] | set([name2id[st] for st in id2inferred_stance[id]])
    #print(graph) # {...100004: {2000826, 3, 1000018, 2001366}, 100005: {2000472, 0, 1000018, 3},....}
    
    '''
    user to @mention user from our data
    '''
    
    for id in name2utype:
        #print('id', id)
        
        #print('name2id[id]', name2id[id])
        for um in name2mention[id]:
            #print('ty', ty)
            if um:
                #print('hi')
                
                mention=name2id[um] 
                
                graph[name2id[id]]=graph[name2id[id]] | set([mention])
    #print(graph)         
                
    #////uncomment
    
    f1=open('data/graph_embd_yoga/yoga_graph_des_net_em2.mapping', 'w')
    f2=open('data/graph_embd_yoga/yoga_graph_des_net_em2.adjlist', 'w')

    all_ids=sorted(id for id in id2name)

    for id in all_ids:
        f1.write(str(id)+"\t"+str(id2name[id])+"\n")
        f2.write(str(id)+"\t")
        adjlist=sorted(list(graph[id]))

        for adj in range(len(adjlist)):
            if adj==0:
                f2.write(str(adjlist[adj]))
            else:
                f2.write(" "+str(adjlist[adj]))
        f2.write("\n")

    f1.close()
    f2.close()
        #print(id2text)
    
        
    twt2tokenized_text, weights_matrix=Tokenize_sentences(name2text, name2id)

    '''
     Create validation set using our manually annotated ground truth
    '''
    #ad2annotated_stance={}
    gt_user = {}
    #print("name2utype_gt", name2utype_gt)
    for id in name2utype_gt:
        #print(id)
        #print('name2utype_gt[id]',name2utype_gt[id], type(name2utype_gt[id]))
        #if id2stance[id][0] != '' and id not in annotated_ads and int(id2stance[id][0])!=-1:
        #if id2stance[id][0] != '' and int(id2stance[id][0]) != -1:
        if name2utype_gt[id] != '' and name2utype_gt[id] != '2':
            #gt_utypes=[name2id[reverse_utypes[gt]] for gt in name2utype_gt[id]]
            gt_utypes= name2id[reverse_utypes[name2utype_gt[id]]]
            #print('gt_utypes', gt_utypes, type(gt_utypes))
            gt_user[name2id[id]]=gt_utypes
    print ("gt_user", len(gt_user)) #786


    with open('data/graph_embd_yoga/data_des_@mention_em2.pickle', 'wb') as out_file:
        pickle.dump([graph, id2name, name2id, type_start, type_end, user_start, user_end, des_start, des_end, \
                     annotated_des, annotated_user, twt2tokenized_text, weights_matrix, \
                     name2text, gt_user], out_file)


    


In [ ]:
Create_Graph() #create graph and validation set, glove weight

In [ ]:
#Count number of edges in graph from adjacency list
f_in =open('data/graph_embd_yoga/yoga_graph_des_net_em2.adjlist', 'r')

Lines = f_in.readlines()
 
edges = 0
nodes = 0
for line in Lines:
    clean_line = line.replace('\n','')
    clean_line = clean_line.replace('\t','')
    clean_line = clean_line.replace('\r','')
    edge_no_perline = len(clean_line.split(' ')) - 1 #edge number per line
    edges = edges + edge_no_perline
    nodes = nodes + 1
    #print(count)
print("total number of nodes : ", nodes) #33784
print("total number of edges : ", edges) # 19927